# Shopping cart analysis and recomender system based on ARM (Association Rule Mining) 
------------------
## TASK I: Data Preprocessing

In this task, we will preprocess the provided e-commerce dataset by cleaning and filtering the data. This involves removing missing values, filtering out negative or zero-priced items, and ensuring that each order contains more than one product. We will also randomly select 20,000 unique users and their corresponding orders for further analysis.

1. **Step 1: Loading the Data**: 
   In this step, we load all the necessary CSV files into DataFrames using `pandas.read_csv()` to access the e-commerce dataset.
   - Files: aisles.csv, departments.csv, products.csv, order_products_train.csv, orders.csv, order_products_prior.csv.

2. **Step 2: Preprocessing the Data**: 
   We will clean the dataset by performing the following actions:
   - **Removing missing values**: Using `dropna()` to eliminate any rows with missing data.
   - **Filtering negative or zero-priced items**: We will filter out any rows where the price is less than or equal to 0.
   - **Filtering out negative quality items**: Removing items with 'Negative' quality value if the column exists.

3. **Step 3: Filtering Single-Item Orders**: 
   In this step, we filter out orders where only one item was purchased. This is done by grouping the data by `InvoiceNo` and only keeping those orders where the count of items is greater than 1.

4. **Step 4: Selecting 20,000 Users and One Order Per User**: 
   - We randomly select 20,000 unique users from the dataset (if contains more than 20,000 unique), and then filter the orders to include only those belonging to the selected users.

5. **Step 5: Saving the Cleaned Data**: 
   After the preprocessing steps are completed, the cleaned data is saved into two separate CSV files:
   - **20000_sampled_orders.csv**: Contains the orders of the randomly selected 20,000 users.
   - **cleaned_data.csv**: Contains all the cleaned data from the various files combined.

In [4]:
import os
import pandas as pd
import dask.dataframe as dd

# Loading data from CSV files
aisles = dd.read_csv('aisles.csv')
orders = dd.read_csv('orders.csv')
products = dd.read_csv('products.csv')
departments = dd.read_csv('departments.csv')
order_products_train = dd.read_csv('order_products__train.csv')
order_products_prior = dd.read_csv('order_products__prior.csv')

ImportError: An error occurred while calling the read_csv method registered to the pandas backend.
Original Message: pyarrow>=10.0.1 is required for PyArrow backed StringArray.

In [ ]:
# Perform operations similar to pandas, but Dask will process in chunks
aisles_departments = dd.merge(aisles, departments, how='left', left_on='aisle_id', right_on='department_id')
products_full = dd.merge(aisles_departments, products, how='left', on='aisle_id')

# Merge order_products_train and order_products_prior
order_products_train_full = dd.merge(order_products_train, products_full, how='left', on='product_id')
order_products_prior_full = dd.merge(order_products_prior, products_full, how='left', on='product_id')

# Final merge with orders
orders_full = dd.merge(orders, order_products_train_full, how='left', on='order_id')
orders_full = dd.merge(orders_full, order_products_prior_full, how='left', on='order_id')

MemoryError: Unable to allocate 247. MiB for an array with shape (32434489,) and data type int64

In [ ]:
# Data preprocessing (removing null values)
def remove_nulls(df):
    df_cleaned = df.dropna()
    return df_cleaned

# Delete invoices (orders) that have only one item
def filter_single_item_invoices(df):
    df_filtered = df.groupby('order_id').filter(lambda x: len(x['product_id']) > 1) 
    return df_filtered

# Filtering orders that have only one item
orders_cleaned = remove_nulls(orders_full)
orders_cleaned = filter_single_item_invoices(orders_cleaned)

# Selecting 20,000 users and one order from each user
unique_users = orders_cleaned['user_id'].drop_duplicates()
sampled_users = unique_users.sample(n=20000, random_state=42)

print(f"Total unique users: {len(unique_users)}")
print(f"Total sampled users: {len(sampled_users)}")

MemoryError: Unable to allocate 985. MiB for an array with shape (4, 32277741) and data type int64

In [ ]:
# Filter aisles, departments, and products data based on the selected 20,000 users' data
aisles_sampled = aisles_cleaned[aisles_cleaned['aisle_id'].isin(sampled_users)]
products_sampled = products_cleaned[products_cleaned['product_id'].isin(sampled_users)]
departments_sampled = departments_cleaned[departments_cleaned['department_id'].isin(sampled_users)]

# First get the `order_id` values from the `orders_cleaned` dataframe that belong to sampled users
orders_sampled = orders_cleaned[orders_cleaned['user_id'].isin(sampled_users)]
sampled_order_ids = orders_sampled['order_id']

# Filter the order_products_train and order_products_prior data based on the sampled `order_id` and the selected 20,000 users
order_products_train_sampled = order_products_train_full[order_products_train_full['order_id'].isin(sampled_order_ids)]
order_products_prior_sampled = order_products_prior_full[order_products_prior_full['order_id'].isin(sampled_order_ids)]

# Save the cleaned data to a final CSV file
final_cleaned_data = pd.concat([aisles_sampled, departments_sampled, products_sampled, 
                                orders_sampled, order_products_train_sampled, order_products_prior_sampled])

output_folder = './output_directory'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
final_cleaned_data.to_csv(f'{output_folder}/cleaned_data.csv', index=False, mode='w')

## TASK II: Base Demand Calculation and Initial Strategy

In this task, we calculate the base demand for each product, define an initial pricing, advertising strategy and Demand and Profit functions. This includes:

1. **Calculate base demand**: The base demand for each product is calculated as the total quantity divided by the number of active months. If no active months are available, total quantity is used.
2. **Assign cost**: Based on the median price of the products, we assign a cost of 5 or 10 units.
3. **Define initial strategy**: The initial price is set to the average price, and advertising budget is initialized.
4. **Save results**: The results are saved for the next simulation stage.
5. **Demand Function (demand_func)**: Calculates the demand based on price, average price of other sellers, advertising budget, and social influence.
6. **Profit Function (profit_func)**: Calculates the profit for each seller based on the demand, price, and advertising budget.

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

# Load the cleaned data and extract only the necessary columns related to 'order_id' and 'product_id'
cleaned_data = pd.read_csv(f'{output_folder}/cleaned_data.csv')
order_products_train_cleaned = cleaned_data[['order_id', 'product_id']]  # Only the relevant columns

# Group by `order_id` to create a list of products for each order
grouped_orders = order_products_train_cleaned.groupby('order_id')['product_id'].apply(list).reset_index(name='product_list')

# Create one-hot encoding for the products in each order using TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit_transform(grouped_orders['product_list'])
one_hot_encoded_df = pd.DataFrame(te_ary, columns=te.columns_)

# Display the first few rows of and save the One-Hot Encoded table to verify the transformation
print(one_hot_encoded_df.head())
one_hot_encoded_df.to_csv(f'{output_folder}/one_hot_encoded_df.csv', index=False)

C:\Users\KAVOSH\AppData\Local\Temp\ipykernel_7100\161614272.py:5: DtypeWarning: Columns (1,3,5,8) have mixed types. Specify dtype option on import or set low_memory=False.
  cleaned_data = pd.read_csv(f'{output_folder}/cleaned_data.csv')
C:\Users\KAVOSH\AppData\Local\Temp\ipykernel_7100\161614272.py:14: FutureWarning: Allowing arbitrary scalar fill_value in SparseDtype is deprecated. In a future version, the fill_value must be a valid value for the SparseDtype.subtype.
  one_hot_encoded_df = pd.DataFrame.sparse.from_spmatrix(te_ary, columns=te.columns_)  # convert sparse matrix to a DataFrame


   1.0   2.0   3.0   4.0   5.0   NaN   NaN   8.0   9.0   10.0  ...  NaN   \
0     0     0     0     0     0     0     0     0     0     0  ...     0   
1     0     0     0     0     0     0     0     0     0     0  ...     0   
2     0     0     0     0     0     0     0     0     0     0  ...     0   
3     0     0     0     0     0     0     0     0     0     0  ...     0   
4     0     0     0     0     0     0     0     0     0     0  ...     0   

   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
0     0     0     0     0     0     0     0     0     0  
1     0     0     0     0     0     0     0     0     0  
2     0     0     0     0     0     0     0     0     0  
3     0     0     0     0     0     0     0     0     0  
4     0     0     0     0     0     0     0     0     0  

[5 rows x 356257 columns]


KeyboardInterrupt: 